# The aHB circuit tracks fictive trajectories

We have seen in the previous notebooks that the aHB integrator could be capabable of keep track of past motion over a significant amount of time. In this notebook, we will see how much it can keep track of past trajectories over significant amounts of time.

## A first look at cumulative directional motion and network phase

In [ ]:
%matplotlib widget
from pathlib import Path

import lotr.plotting as pltltr
import numpy as np
import pandas as pd
from bouter.utilities import crop
from lotr import A_FISH, LotrExperiment, dataset_folders
from lotr.behavior import get_bouts_props_array
from lotr.default_vals import REGRESSOR_TAU_S
from lotr.utils import convolve_with_tau, linear_regression, nan_phase_jumps
from matplotlib import pyplot as plt
from scipy.stats import spearmanr, wilcoxon
from tqdm import tqdm

COLS = pltltr.COLS

In [ ]:
# First, get the unwrapped phase:
exp = LotrExperiment(A_FISH)
network_phase = np.unwrap(exp.network_phase)

# Now, let's find a way of reconstruct past motion of the fish.
# We will start with a vector of zeros that has ones at timepoints
# where the fish swam:
theta_turned = get_bouts_props_array(
    exp.n_pts, exp.bouts_df, selection="all", value="bias",
)

# Fictive heading will be the cumulative sum of this array:
fictive_head = np.cumsum(theta_turned)

# Finally, we smooth this array with a kernel that matches the slower dynamics
# of the neuron (results are not much affected by this)
fictive_head = convolve_with_tau(np.cumsum(theta_turned), REGRESSOR_TAU_S * exp.fn)

In [ ]:
f, axs = plt.subplots(
    3, 1, figsize=(5, 3), sharex=True, gridspec_kw=(dict(bottom=0.12))
)
lw = 1

labels = ["Tail angle", "Angle turned (dθ)", "Fict. heading (θ)"]
col_seq = [COLS["beh"], pltltr.shift_lum(COLS["th_plot"], 0.2), COLS["th_plot"]]
for i, (x, y) in enumerate(
    [
        (exp.behavior_log["t"], exp.behavior_log["tail_sum"]),
        (exp.time_arr, theta_turned),
        (exp.time_arr, fictive_head),
    ]
):
    axs[i].plot(
        x, y, lw=lw, c=col_seq[i], label=labels[i], rasterized=i == 0
    )  # rasterize for exporting
    axs[i].legend(loc=1, bbox_to_anchor=(1.1, 1.1))

axs[0].set(**pltltr.get_pi_labels(1, ax="y"))
axs[1].set(**pltltr.get_pi_labels(0.5, ax="y"), ylim=(-1.8, 1.8))
axs[2].set(xlabel="Time (s)", **pltltr.get_pi_labels(coefs=(0, -5, -10, -15), ax="y"))

[pltltr.despine(ax) for ax in axs.flatten()]
plt.show()
pltltr.savefig("fictive_traj_computation")

Now, how does the network phase look, compared with the fictive heading?

In [ ]:
t_off_s, t_dur_s = 150, 400

f, ax = plt.subplots(
    1, 1, figsize=(4, 1.5), gridspec_kw=(dict(bottom=0.25, right=0.85))
)
twin_ax = ax.twinx()
ax.plot(
    exp.time_arr, fictive_head, c=COLS["th_plot"],
)
twin_ax.plot(
    exp.time_arr, np.unwrap(exp.network_phase), c=COLS["ph_plot"],
)
ax.set_ylabel("Fict. heading (θ)", c=COLS["th_plot"])
ax.set(xlabel="Times (s)", **pltltr.get_pi_labels(coefs=(0, -5, -10, -15, -20), ax="y"))
twin_ax.set_ylabel("Network phase (ϕ)", c=COLS["ph_plot"])
twin_ax.set(**pltltr.get_pi_labels(coefs=(0, 2, 4, 6), ax="y"))
ax.set(ylim=(2, -65))
# rho, pval = spearmanr(fictive_head, np.unwrap(exp.network_phase))
# ax.text(1200, -np.pi, "$ρ_r=" + f"{rho:0.3f}" + "$" + pltltr.get_pval_stars(pval))
[pltltr.despine(a, ["top"]) for a in [ax, twin_ax]]
ax.axvspan(t_off_s, t_off_s + t_dur_s, lw=0, fc=".9", zorder=-100)
pltltr.savefig("fictive_traj_phase_onefish")

## Example of sorted traces and behavior

In [ ]:
sorted_traces = exp.traces[:, exp.hdn_indexes][:, np.argsort(exp.rpc_angles)]

In [ ]:
pts_slice = slice(t_off_s * exp.fn, (t_off_s + t_dur_s) * exp.fn)
beh_seg = exp.behavior_log[
    (exp.behavior_log["t"] > t_off_s) & (exp.behavior_log["t"] < (t_off_s + t_dur_s))
]
f, axs = plt.subplots(
    2,
    1,
    figsize=(4.5, 2),
    gridspec_kw=dict(height_ratios=(0.3, 1), hspace=0, left=0.2, bottom=0.2, right=0.8),
)
axs[0].plot(beh_seg["t"] - t_off_s, beh_seg["tail_sum"], c=COLS["beh"], lw=1)
im = axs[1].imshow(
    sorted_traces[pts_slice, :].T,
    aspect="auto",
    extent=(0, t_dur_s, -np.pi, np.pi),
    cmap=COLS["dff_plot"],
    vmin=-1.7,
    vmax=1.7,
)
norm_phase = nan_phase_jumps(exp.network_phase)
norm_head = nan_phase_jumps(np.mod(exp.fictive_heading / 2.2 + 0.5, 2 * np.pi)) - np.pi
axs[1].plot([np.nan], [np.nan], c=COLS["beh"], label="tail")
axs[1].plot(
    exp.time_arr[pts_slice] - t_off_s,
    -norm_head[pts_slice],
    c=COLS["th_plot"],
    label="heading (θ)",
)
axs[1].plot(
    exp.time_arr[pts_slice] - t_off_s,
    norm_phase[pts_slice],
    c=COLS["ph_plot"],
    label="phase (ϕ)",
)

pltltr.add_scalebar(
    ax=axs[0], ylen=np.pi, xlen=0, xpos=0, ypos=0, xlabel="", ylabel="π", fontsize=6,
)
axs[0].set(
    ylim=(-3.8, 3.8), xlim=(0, t_dur_s),
)
axs[1].set(
    xlim=(0, t_dur_s),
    xlabel="Time (s)",
    ylabel="ROI angle (approx.)",
    **pltltr.get_pi_labels(0.5, ax="y")
)
axs[1].legend(
    bbox_to_anchor=(0.95, 1.3),
    fontsize=8,
    handlelength=0.0,
    labelspacing=0.3,
    labelcolor="linecolor",
)
pltltr.add_dff_cbar(im, axs[1], (1.05, 0.1, 0.02, 0.2))
pltltr.despine(axs[1])
pltltr.despine(axs[0], "all")

pltltr.savefig("fictive_traj_phase_onefish_zoom_traces")

In [ ]:
t_off_s, t_dur_s = 150, 400

pts_slice = slice(t_off_s * exp.fn, (t_off_s + t_dur_s) * exp.fn)
beh_seg = exp.behavior_log[
    (exp.behavior_log["t"] > t_off_s) & (exp.behavior_log["t"] < (t_off_s + t_dur_s))
]
f, axs = plt.subplots(
    2,
    1,
    figsize=(4.5, 3),
    gridspec_kw=dict(height_ratios=(0.3, 1), hspace=0, left=0.2, bottom=0.2, right=0.8),
)
axs[0].plot(beh_seg["t"] - t_off_s, beh_seg["tail_sum"], c=COLS["beh"], lw=1)
im = axs[1].imshow(
    np.concatenate([sorted_traces[pts_slice, :],] * 3, 1).T,
    aspect="auto",
    extent=(0, t_dur_s, -3 * np.pi, 3 * np.pi),
    cmap=COLS["dff_plot"],
    vmin=-1.7,
    vmax=1.7,
)
norm_phase = exp.network_phase
norm_head = exp.fictive_heading / 2.2 + 0.5 + 3 * np.pi
axs[1].plot([np.nan], [np.nan], c=COLS["beh"], label="tail")
axs[1].plot(
    exp.time_arr[pts_slice] - t_off_s,
    -norm_head[pts_slice],
    c=COLS["th_plot"],
    label="heading (θ)",
)
axs[1].plot(
    exp.time_arr[pts_slice] - t_off_s,
    np.unwrap(norm_phase[pts_slice]),
    c=COLS["ph_plot"],
    label="phase (ϕ)",
)

pltltr.add_scalebar(
    ax=axs[0], ylen=np.pi, xlen=0, xpos=0, ypos=0, xlabel="", ylabel="π", fontsize=6,
)
axs[0].set(
    ylim=(-3.8, 3.8), xlim=(0, t_dur_s),
)
axs[1].set(
    xlim=(0, t_dur_s),
    xlabel="Time (s)",
    ylabel="ROI angle (approx.)",
    **pltltr.get_pi_labels(0.5, ax="y")
)
axs[1].legend(
    bbox_to_anchor=(1.3, 1.3),
    fontsize=8,
    handlelength=0.0,
    labelspacing=0.3,
    labelcolor="linecolor",
)
pltltr.add_dff_cbar(im, axs[1], (1.05, 0.1, 0.02, 0.2))
pltltr.despine(axs[1])
pltltr.despine(axs[0], "all")
pltltr.savefig("fictive_traj_phase_onefish_zoom_traces_v2")

## Quantify correlation over the entire dataset

To run the correlation over the entire dataset, we will not use the whole trace to avoid the problem that a large drift at some point of the experiment could disrupt the correlation for the rest of it. Instead, we will use correlation in windows of 5 minutes, including only times where there were at least 2 directional bouts happening (otherwise fictive heading will be a flat line, with which correlation ill-defined).

In [ ]:
CORR_WND_S = 300  # window over which correlation will be computed
N_OVERLAPS = 10  # number of overlaps per window
MIN_BOUTS = 2  # minimum number of bouts to inclide in the window

results_df = []
for path in tqdm(dataset_folders):
    # for path in tqdm(maybe):
    exp = LotrExperiment(path)
    phase = np.unwrap(exp.network_phase)
    theta_turned = get_bouts_props_array(
        exp.n_pts, exp.bouts_df, selection="all", value="bias",
    )

    # Fictive heading will be the cumulative sum of this array:
    fictive_head = np.cumsum(theta_turned)
    wnd_pts = int(CORR_WND_S * exp.fn)
    overlap_pts = wnd_pts // N_OVERLAPS

    # Find what ranges are valid, i.e have enough bouts in the interval:
    indexes = list(range(0, len(phase) - wnd_pts, overlap_pts))
    valid_indexes = []
    for i in indexes:
        if np.sum(np.abs(theta_turned[i : i + wnd_pts])) > MIN_BOUTS:
            valid_indexes.append(i)

    shuf_indexes = []
    for i in valid_indexes:
        shuf_i = i
        c = 0
        while np.abs(i - shuf_i) < wnd_pts // 2:
            shuf_i = np.random.choice(valid_indexes)
            c += 1
            if c > 50:
                print("could not find a valid shuffle for ", path.name)
                break

        shuf_indexes.append(shuf_i)

    for i, shuf_i in zip(valid_indexes, shuf_indexes):
        wnd = slice(i, i + wnd_pts)
        shuf_wnd = slice(shuf_i, shuf_i + wnd_pts)

        toshuf = fictive_head[shuf_wnd]

        if np.random.rand() > 0.5:
            toshuf = -toshuf

        rho, _ = spearmanr(phase[wnd], fictive_head[wnd])
        rho_shuf, _ = spearmanr(phase[wnd], toshuf)

        results_df.append(dict(rho=rho, shuf=rho_shuf, fid=exp.dir_name))

In [ ]:
res_df = pd.DataFrame(results_df)
# compute quantiles:
quantiles_df = res_df.groupby("fid").quantile((0.25, 0.50, 0.75))

# sort on 0.5 quantile:
sort_order = quantiles_df.xs(0.5, level=1).sort_values("rho").index

# compute p values for each fish:
def _wilcoxon_p(df, keys=["rho", "shuf"]):
    _, p = wilcoxon(*[df[k] for k in keys])
    return p


pvals_df = res_df.groupby("fid").apply(_wilcoxon_p)

In [ ]:
f, ax = plt.subplots(figsize=(3, 2), gridspec_kw=dict(bottom=0.2, left=0.18),)
for k, lab, col in zip(
    ["shuf", "rho"], ["Shuffle", "Data"], [COLS["shuf"], COLS["ph_plot"]]
):
    stacked_quantiles = quantiles_df[[k]].stack().unstack(level=0)
    pltltr.tick_with_bars(
        stacked_quantiles[sort_order], cols=[col,] * len(sort_order), s=0.2,
    )
    # Dummy plot for legend:
    plt.plot([np.nan], [np.nan], c=col, label=lab)

y_txt = 0.7
for i, pval in enumerate(pvals_df[sort_order]):
    plt.text(
        i,
        y_txt,
        pltltr.get_pval_stars(pval),
        fontsize=6,
        rotation="vertical",
        zorder=100,
        va="top",
    )

ax.legend(loc=4, fontsize=7)
# plt.plot(rhos[np.argsort(rhos[:, 1]), :], c="r")
ax.set(xlabel="Fish (sorted by corr.)", ylabel="Heading/phase corr. ($ρ_r$)")
ax.axhline(0, lw=1, c=".4")
pltltr.despine(ax)
plt.show()
pltltr.savefig("correlations_all_fish")